In [2]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('C:/Users/Onwer/Downloads/vehicles.csv')

In [4]:
# Getting info on type 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [5]:
# Calculating total no. of null values in each column as we saw there multiple nan values in columns
df.isnull().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [6]:
#Dropping the columns which is not usable or have high percentage of null values in order to reduce the size of data 
# which will decrease the processing time as well
df.drop(columns=['id','url','region_url','VIN','image_url','description','county','posting_date','size','lat','long'],inplace=True)

In [7]:
# Count duplicate rows
duplicate_count = df[df.duplicated()].shape[0]
# Print the count of duplicate rows
print("Number of duplicate rows:", duplicate_count)

Number of duplicate rows: 56491


In [8]:
# Drop duplicate rows
df = df.drop_duplicates()

In [9]:
##imputing values in null cells of manufacturer column based on the corresponding model of the car
# Step 1: Rearrange the 'model' column
df['model'] = sorted(df['model'], key=lambda x: (not str(x).isdigit(), str(x)))

# Step 2: Split the 'model' column and keep only the first value
df['model'] = df['model'].str.split().str[0]

# Step 3: Filter and show only the unique values
unique_models = df['model'].unique()
print(unique_models)

['0' '00000' '00000000000000' ... '♿' '𝓜𝓮𝓻𝓬𝓮𝓭𝓮𝓼' '🔥GMC']


In [10]:
#creating a variable which contains all the rows with null values in manufacturer column
#which we can impute based on the model column
null_manufacturer_rows = df[df['manufacturer'].isnull()]

In [11]:
# Define a mapping dictionary with models as keys and manufacturers as values
model_to_manufacturer = {
    '200sl': 'Mercedes-Benz',
    '228i': 'BMW',
    '230i': 'BMW',
    '240d': 'Mercedes-Benz',
    '240z': 'Datsun',
    '2500hd': 'GMC',
    '280zx': 'Datsun',
    '3-series': 'BMW',
    '300-series': 'Lexus',
    '3000gt': 'Mitsubishi',
    '300zx': 'Nissan',
    '320i': 'BMW',
    '325i': 'BMW',
    '325is': 'BMW',
    '328i': 'BMW',
    '328xi': 'BMW',
    '330ci': 'BMW',
    '330i': 'BMW',
    '330xi': 'BMW',
    '335i': 'BMW',
    '3500hd': 'GMC',
    '350z': 'Nissan',
    '370z': 'Nissan',
    '4-runner': 'Toyota',
    '428i': 'BMW',
    '430i': 'BMW',
    '450gl': 'Mercedes-Benz',
    '450slc': 'Mercedes-Benz',
    '4runner': 'Toyota',
    '5-series': 'BMW',
    '500c': 'Fiat',
    '500sl': 'Mercedes-Benz',
    '500x': 'Fiat',
    '525i': 'BMW',
    '528i': 'BMW',
    '528xi': 'BMW',
    '530i': 'BMW',
    '535i': 'BMW',
    '540i': 'BMW',
    '550i': 'BMW',
    '645ci': 'BMW',
    '650i': 'BMW',
    '7-series': 'BMW',
    '740li': 'BMW',
    '750li': 'BMW',
    'AM': 'Aston Martin',
    'AMC': 'AMC',
    'BMX': 'BMW',
    'Bentley': 'Bentley',
    'Blue': 'Blue Bird',
    'CHASSIS': 'CHASSIS',
    'CHEC': 'CHEC',
    'CHECKER': 'Checker',
    'CHEV.': 'Chevrolet',
    'CHOVROLET': 'Chevrolet',
    'Corolla': 'Toyota',
    'E350': 'Mercedes-Benz',
    'ELANTRA': 'Hyundai',
    'ELEMENT': 'Honda',
    'ES': 'Lexus',
    'Escape': 'Ford',
    'F-150': 'Ford',
    'F-250': 'Ford',
    'F0RD': 'Ford',
    'F150': 'Ford',
    'F450': 'Ford',
    'FORD/MUSTANG': 'Ford',
    'Foed': 'Ford',
    'For': 'Ford',
    'Ford,': 'Ford',
    'Freightliner': 'Freightliner',
    'GEM': 'GEM',
    'Genesis': 'Hyundai',
    'HUMMER': 'Hummer',
    'Hino': 'Hino',
    'IHC': 'International Harvester',
    'INTERNATIONAL': 'International Harvester',
    'Internatioal': 'International Harvester',
    'International': 'International Harvester',
    'Isuzu': 'Isuzu',
    'Lamborghini': 'Lamborghini',
    'MB': 'Mercedes-Benz',
    'MINI/COOPER': 'MINI',
    'Maserati': 'Maserati',
    'Mazda3': 'Mazda',
    'Mercede-Benz': 'Mercedes-Benz',
    'MiniCooper': 'MINI',
    'Mustang': 'Ford',
    'OLDSMOBILE': 'Oldsmobile',
    'Oldsmobile': 'Oldsmobile',
    'PETERBILT': 'Peterbilt',
    'PT': 'Chrysler',
    'Plymouth': 'Plymouth',
    'Pontica': 'Pontiac',
    'Rolls': 'Rolls-Royce',
    'STERLING': 'Sterling',
    'Saab': 'Saab',
    'Scion': 'Toyota',
    'Suabru': 'Subaru',
    'Subaro': 'Subaru',
    'Town': 'Chrysler',
    'Triumph': 'Triumph',
    'WANTED!': 'WANTED!',
    'WPOAA2993XS620614': 'Porsche',
    'Workhorse': 'Workhorse',
    'XLT': 'Ford',
    'Yamaha': 'Yamaha',
    'a-class': 'Mercedes-Benz',
    'a3': 'Audi',
    'a4': 'Audi',
    'a5': 'Audi',
    'a6': 'Audi',
    'a7': 'Audi',
    'a8': 'Audi',
    'acadia': 'GMC',
    'accent': 'Hyundai',
    'accord': 'Honda',
    'acty': 'Honda',
    'aerostar': 'Ford',
    'alanta': 'Chevrolet',
    'all-new': 'Mitsubishi',
    'allroad': 'Audi',
    'altima': 'Nissan',
    'amanti': 'Kia',
    'amc': 'AMC',
    'armada': 'Nissan',
    'arteon': 'Volkswagen',
    'ascent': 'Subaru',
    'astra': 'Saturn',
    'astro': 'Chevrolet',
    'atlas': 'Volkswagen',
    'ats': 'Cadillac',
    'aura': 'Saturn',
    'avalanche': 'Chevrolet',
    'avalon': 'Toyota',
    'avenger': 'Dodge',
    'aveo': 'Chevrolet',
    'azera': 'Hyundai',
    'b3000': 'Mazda',
    'b4000': 'Mazda',
    'baja': 'Subaru',
    'beetle': 'Volkswagen',
    'beetle-classic': 'Volkswagen',
    'bel': 'Chevrolet',
    'belair': 'Chevrolet',
    'benz': 'Mercedes-Benz',
    'blazer': 'Chevrolet',
    'bolt': 'Chevrolet',
    'bonanza': 'Chevrolet',
    'bonneville': 'Pontiac',
    'boxster': 'Porsche',
    'bronco': 'Ford',
    'brz': 'Subaru',
    'c': 'Chrysler',
    'c-10': 'Chevrolet',
    'c-class': 'Mercedes-Benz',
    'c-hr': 'Toyota',
    'c-max': 'Ford',
    'c/k': 'Chevrolet',
    'c10': 'Chevrolet',
    'c230': 'Mercedes-Benz',
    'c280': 'Mercedes-Benz',
    'c300': 'Mercedes-Benz',
    'c4500': 'GMC',
    'c5500': 'GMC',
    'cabellero': 'GMC',
    'cabriolet': 'Volkswagen',
    'caddilac': 'Cadillac',
    'cadenza': 'Kia',
    'camaro': 'Chevrolet',
    'camry': 'Toyota',
    'canyon': 'GMC',
    'capri': 'Ford',
    'caprice': 'Chevrolet',
    'captiva': 'Chevrolet',
    'caravan': 'Dodge',
    'cargo': 'Chevrolet',
    'cavalier': 'Chevrolet',
    'cayenne': 'Porsche',
    'cayman': 'Porsche',
    'cc': 'Volkswagen',
    'cc4500': 'Chevrolet',
    'celica': 'Toyota',
    'centra': 'Nissan',
    'century': 'Buick',
    'challenger': 'Dodge',
    'challengers': 'Dodge',
    'charger': 'Dodge',
    'cherokee': 'Jeep',
    'chevelle': 'Chevrolet',
    'chevorlet': 'Chevrolet',
    'chryler': 'Chrysler',
    'civic': 'Honda',
    'cj5': 'Jeep',
    'cl': 'Acura',
    'cl-class': 'Mercedes-Benz',
    'cla': 'Mercedes-Benz',
    'cla-class': 'Mercedes-Benz',
    'cla45': 'Mercedes-Benz',
    'clarity': 'Honda',
    'clk': 'Mercedes-Benz',
    'clk-class': 'Mercedes-Benz',
    'clk320': 'Mercedes-Benz',
    'clk350': 'Mercedes-Benz',
    'clk550': 'Mercedes-Benz',
    'cls': 'Mercedes-Benz',
    'cls-class': 'Mercedes-Benz',
    'club': 'Mercedes-Benz',
    'cmax': 'Ford',
    'cobalt': 'Chevrolet',
    'colbalt': 'Chevrolet',
    'colorado': 'Chevrolet',
    'commander': 'Jeep',
    'compass': 'Jeep',
    'continental': 'Lincoln',
    'convertible': 'BMW',
    'cooper': 'MINI',
    'corolla': 'Toyota',
    'corolla..le': 'Toyota',
    'corsair': 'Lincoln',
    'corvair': 'Chevrolet',
    'corvette': 'Chevrolet',
    'cougar': 'Mercury',
    'countryman': 'MINI',
    'coupe': 'BMW',
    'cr-v': 'Honda',
    'cr-z': 'Honda',
    'crossfire': 'Chrysler',
    'crosstrek': 'Subaru',
    'crown': 'Toyota',
    'cruse': 'Chevrolet',
    'cruze': 'Chevrolet',
    'crv': 'Honda',
    'ct': 'Lexus',
    'ct4': 'Cadillac',
    'ct5': 'Cadillac',
    'ct6': 'Cadillac',
    'cts': 'Cadillac',
    'cts-v': 'Cadillac',
    'ctsv': 'Cadillac',
    'cube': 'Nissan',
    'cummin': 'Cummins',
    'cummins': 'Cummins',
    'cv': 'Chevrolet',
    'cx-3': 'Mazda',
    'cx-5': 'Mazda',
    'cx-7': 'Mazda',
    'cx-9': 'Mazda',
    'cx9': 'Mazda',
    'dakota': 'Dodge',
    'dart': 'Dodge',
    'deisel': 'Diesel',
    'deluxe': 'Ford',
    'deville': 'Cadillac',
    'diesel': 'Diesel',
    'discovery': 'Land Rover',
    'dts': 'Cadillac',
    'durango': 'Dodge',
    'e': 'Mercedes-Benz',
    'e-150': 'Ford',
    'e-250': 'Ford',
    'e-350': 'Ford',
    'e-450': 'Ford',
    'e-class': 'Mercedes-Benz',
    'e-golf': 'Volkswagen',
    'e-pace': 'Jaguar',
    'e-series': 'Ford',
    'e150': 'Ford',
    'e250': 'Ford',
    'e350': 'Ford',
    'e450': 'Ford',
    'e550': 'Ford',
    'eagle': 'Eagle',
    'echo': 'Toyota',
    'eclipse': 'Mitsubishi',
    'eclipses': 'Mitsubishi',
    'econoline': 'Ford',
    'ecosport': 'Ford',
    'edge': 'Ford',
    'el': 'Chevrolet',
    'elantra': 'Hyundai',
    'eldorado': 'Cadillac',
    'element': 'Honda',
    'enclave': 'Buick',
    'encore': 'Buick',
    'endeavor': 'Mitsubishi',
    'entourage': 'Hyundai',
    'envoy': 'GMC',
    'envoy/sle': 'GMC',
    'eos': 'Volkswagen',
    'equinox': 'Chevrolet',
    'equus': 'Hyundai',
    'es300': 'Lexus',
    'es330': 'Lexus',
    'es350': 'Lexus',
    'escalade': 'Cadillac',
    'escape-se,': 'Ford',
    'eurovan': 'Volkswagen',
    'evoque': 'Land Rover',
    'ex35': 'Infiniti',
    'excursion': 'Ford',
    'expedition': 'Ford',
    'explorer': 'Ford',
    'express': 'Chevrolet',
    'f': 'Ford',
    'f-150': 'Ford',
    'f-250': 'Ford',
    'f-250sd': 'Ford',
    'f-350': 'Ford',
    'f-350sd': 'Ford',
    'f-450': 'Ford',
    'f-450sd': 'Ford',
    'f-550': 'Ford',
    'f-650': 'Ford',
    'f-750': 'Ford',
    'f-pace': 'Jaguar',
    'f-super': 'Ford',
    'f-superduty': 'Ford',
    'f-type': 'Jaguar',
    'f1': 'Ford',
    'f100': 'Ford',
    'f150': 'Ford',
    'f250': 'Ford',
    'f250sd': 'Ford',
    'f350': 'Ford',
    'f350sd': 'Ford',
    'f450': 'Ford',
    'f550': 'Ford',
    'f650': 'Ford',
    'f650xl': 'Ford',
    'f800': 'Ford',
    'fairlane': 'Ford',
    'falcon': 'Ford',
    'fiesta': 'Ford',
    'fifth': 'Chrysler',
    'firebird': 'Pontiac',
    'fit': 'Honda',
    'fj': 'Toyota',
    'flex': 'Ford',
    'focus': 'Ford',
    'forester': 'Subaru',
    'forte': 'Kia',
    'freestar': 'Ford',
    'freestyle': 'Ford',
    'frontier': 'Nissan',
    'fusion': 'Ford',
    'fuso': 'Mitsubishi',
    'fx35': 'Infiniti',
    'g': 'Infiniti',
    'g35': 'Infiniti',
    'g37': 'Infiniti',
    'g37x': 'Infiniti',
    'g5': 'Pontiac',
    'g6': 'Pontiac',
    'g8': 'Pontiac',
    'genesis': 'Hyundai',
    'geo': 'Geo',
    'gl': 'Mercedes-Benz',
    'gl-class': 'Mercedes-Benz',
    'gla': 'Mercedes-Benz',
    'gla-class': 'Mercedes-Benz',
    'gladiator': 'Jeep',
    'glc': 'Mercedes-Benz',
    'glc-class': 'Mercedes-Benz',
    'gle': 'Mercedes-Benz',
    'glk': 'Mercedes-Benz',
    'glk-class': 'Mercedes-Benz',
    'gls': 'Mercedes-Benz',
    'golf': 'Volkswagen',
    'grand': 'Jeep',
    'grand+cherokee': 'Jeep',
    'gs': 'Lexus',
    'gs300': 'Lexus',
    'gs350': 'Lexus',
    'gs400': 'Lexus',
    'gti': 'Volkswagen',
    'gto': 'Pontiac',
    'gx': 'Lexus',
    'gx470': 'Lexus',
    'hardtop': 'Mercedes-Benz',
    'hhr': 'Chevrolet',
    'highlander': 'Toyota',
    'hr-v': 'Honda',
    'hrv': 'Honda',
    'hyandai': 'Hyundai',
    'hyndai': 'Hyundai',
    'ilx': 'Acura',
    'impala': 'Chevrolet',
    'impreza': 'Subaru',
    'insight': 'Honda',
    'international': 'International Harvester',
    'ion': 'Saturn',
    'ioniq': 'Hyundai',
    'is': 'Lexus',
    'is250': 'Lexus',
    'is350': 'Lexus',
    'isuzu': 'Isuzu',
    'jetta': 'Volkswagen',
    'jimmy': 'GMC',
    'john': 'John Deere',
    'journey': 'Dodge',
    'juke': 'Nissan',
    'justy': 'Subaru',
    'jx': 'Infiniti',
    'k10': 'Chevrolet',
    'k900': 'Kia',
    'karmann': 'Volkswagen',
    'kona': 'Hyundai',
    'l-series': 'Saturn',
    'lacrosse': 'Buick',
    'lancer': 'Mitsubishi',
    'land': 'Land Rover',
    'landcruiser': 'Toyota',
    'leaf': 'Nissan',
    'legacy': 'Subaru',
    'lesabre': 'Buick',
    'liberty': 'Jeep',
    'lifted': 'Chevrolet',
    'limousine': 'Cadillac',
    'liteace': 'Toyota',
    'lr2': 'Land Rover',
    'lr3': 'Land Rover',
    'lr4': 'Land Rover',
    'ls': 'Lexus',
    'ls430': 'Lexus',
    'ls460': 'Lexus',
    'ltd': 'Ford',
    'lucerne': 'Buick',
    'lumina': 'Chevrolet',
    'lx': 'Lexus',
    'm': 'BMW',
    'm-class': 'Mercedes-Benz',
    'm.b.': 'Mercedes-Benz',
    'm3': 'BMW',
    'm35x': 'Infiniti',
    'm37': 'Infiniti',
    'm4': 'BMW',
    'm45': 'Infiniti',
    'm5': 'BMW',
    'm6': 'BMW',
    'macan': 'Porsche',
    'magnum': 'Dodge',
    'malibu': 'Chevrolet',
    'mariner': 'Mercury',
    'mark': 'Lincoln',
    'maserati': 'Maserati',
    'matrix': 'Toyota',
    'maxima': 'Nissan',
    'mazda3': 'Mazda',
    'mazda5': 'Mazda',
    'mazda6': 'Mazda',
    'mazdaspeed6': 'Mazda',
    'mdx': 'Acura',
    'mercedes-amg': 'Mercedes-Benz',
    'mercedez': 'Mercedes-Benz',
    'metris': 'Mercedes-Benz',
    'mg': 'MG',
    'miata': 'Mazda',
    'milan': 'Mercury',
    'mirage': 'Mitsubishi',
    'mkc': 'Lincoln',
    'mks': 'Lincoln',
    'mkt': 'Lincoln',
    'mkx': 'Lincoln',
    'mkz': 'Lincoln',
    'ml': 'Mercedes-Benz',
    'ml350': 'Mercedes-Benz',
    'model': 'Tesla',
    'monte': 'Chevrolet',
    'montego': 'Mercury',
    'montero': 'Mitsubishi',
    'mountaineer': 'Mercury',
    'mr2': 'Toyota',
    'murano': 'Nissan',
    'mustang': 'Ford',
    'mx-5': 'Mazda',
    'nautilus': 'Lincoln',
    'navigator': 'Lincoln',
    'new': 'New Flyer',
    'newport': 'Chrysler',
    'niro': 'Kia',
    'nisaan': 'Nissan',
    'nitro': 'Dodge',
    'nova': 'Chevrolet',
    'nsx': 'Acura',
    'nv': 'Nissan',
    'nv1500': 'Nissan',
    'nv200': 'Nissan',
    'nv2500': 'Nissan',
    'nx': 'Lexus',
    'odyssey': 'Honda',
    'oldsmobile': 'Oldsmobile',
    'optima': 'Kia',
    'outback': 'Subaru',
    'outlander': 'Mitsubishi',
    'outlook': 'Saturn',
    'p900': 'Nissan',
    'pacifica': 'Chrysler',
    'panamera': 'Porsche',
    'park': 'Chevrolet',
    'paseo': 'Toyota',
    'passat': 'Volkswagen',
    'passport': 'Honda',
    'pathfinder': 'Nissan',
    'patriot': 'Jeep',
    'patriot.': 'Jeep',
    'pick': 'Toyota',
    'pickup': 'Toyota',
    'pilot': 'Honda',
    'police': 'Ford',
    'prelude': 'Honda',
    'prius': 'Toyota',
    'probe': 'Ford',
    'promaster': 'Ram',
    'protege': 'Mazda',
    'pt': 'Chrysler',
    'q3': 'Audi',
    'q40': 'Infiniti',
    'q45': 'Infiniti',
    'q5': 'Audi',
    'q50': 'Infiniti',
    'q60': 'Infiniti',
    'q7': 'Audi',
    'q8': 'Audi',
    'quad': 'Polaris',
    'quest': 'Nissan',
    'qx50': 'Infiniti',
    'qx56': 'Infiniti',
    'qx60': 'Infiniti',
    'qx70': 'Infiniti',
    'qx80': 'Infiniti',
    'r': 'Lexus',
    'r-class': 'Mercedes-Benz',
    'rabbit': 'Volkswagen',
    'ram': 'Ram',
    'range': 'Land Rover',
    'ranger': 'Ford',
    'raptor': 'Ford',
    'rav4': 'Toyota',
    'rav4-le': 'Toyota',
    'rav4x': 'Toyota',
    'rdx': 'Acura',
    'red': 'Ford',
    'redline': 'Saturn',
    'regal': 'Buick',
    'regal-t': 'Buick',
    'regency': 'Oldsmobile',
    'relay': 'Saturn',
    'renegade': 'Jeep',
    'ridgeline': 'Honda',
    'rio': 'Kia',
    'riviera': 'Buick',
    'roadster': 'Tesla',
    'rogue': 'Nissan',
    'rogue.': 'Nissan',
    'rogue/select': 'Nissan',
    'rx': 'Lexus',
    'rx300': 'Lexus',
    'rx330': 'Lexus',
    'rx350': 'Lexus',
    'rx450h': 'Lexus',
    's': 'BMW',
    's-class': 'Mercedes-Benz',
    's10': 'Chevrolet',
    's2000': 'Honda',
    's4': 'Audi',
    's5': 'Audi',
    's550': 'Mercedes-Benz',
    's60': 'Volvo',
    's60i': 'Volvo',
    's60t5': 'Volvo',
    's70': 'Volvo',
    's8': 'Audi',
    's80': 'Volvo',
    's90': 'Volvo',
    'sable': 'Mercury',
    'safari': 'GMC',
    'santa': 'Hyundai',
    'santa-fe': 'Hyundai',
    'saturn': 'Saturn',
    'sc': 'Lexus',
    'sc2': 'Saturn',
    'sc400': 'Lexus',
    'sc430': 'Lexus',
    'sedan': 'Toyota',
    'sedona': 'Kia',
    'sequoia': 'Toyota',
    'seven': 'Lotus',
    'shelby': 'Ford',
    'sienna': 'Toyota',
    'sierra': 'GMC',
    'sierra+denali': 'GMC',
    'sierra/sle': 'GMC',
    'silverado': 'Chevrolet',
    'silverado.': 'Chevrolet',
    'sl': 'Mercedes-Benz',
    'sl-class': 'Mercedes-Benz',
    'slc': 'Mercedes-Benz',
    'slc-class': 'Mercedes-Benz',
    'slk': 'Mercedes-Benz',
    'slk-class': 'Mercedes-Benz',
    'sls': 'Mercedes-Benz',
    'sls-amg': 'Mercedes-Benz',
    'sls-class': 'Mercedes-Benz',
    'solstice': 'Pontiac',
    'sonata': 'Hyundai',
    'sonic': 'Chevrolet',
    'sorento': 'Kia',
    'soul': 'Kia',
    'spark': 'Chevrolet',
    'spectra': 'Kia',
    'sportage': 'Kia',
    'sprinter': 'Mercedes-Benz',
    'spyder': 'Porsche',
    'sr5': 'Toyota',
    'ss': 'Chevrolet',
    'sts': 'Cadillac',
    'suburban': 'Chevrolet',
    'sundance': 'Dodge',
    'sunfire': 'Pontiac',
    'super': 'Chevrolet',
    'superduty': 'Ford',
    'supra': 'Toyota',
    'swift': 'Suzuki',
    't100': 'Toyota',
    'tacoma': 'Toyota',
    'tahoe': 'Chevrolet',
    'talon': 'Eagle',
    'taurus': 'Ford',
    'tbd': 'TBD',
    'tco': 'Mercedes-Benz',
    'tempo': 'Ford',
    'terrain': 'GMC',
    'terraza': 'Buick',
    'terrian': 'GMC',
    'tiburon': 'Hyundai',
    'tiguan': 'Volkswagen',
    'titan': 'Nissan',
    'titanium': 'Ford',
    'tl': 'Acura',
    'tlx': 'Acura',
    'touareg': 'Volkswagen',
    'touareg.': 'Volkswagen',
    'town': 'Chrysler',
    'townace': 'Toyota',
    'towncar': 'Lincoln',
    'towncountry': 'Chrysler',
    'trailblazer': 'Chevrolet',
    'transit': 'Ford',
    'transit-connect': 'Ford',
    'transit-350': 'Ford',
    'traverse': 'Chevrolet',
    'trax': 'Chevrolet',
    'tribeca': 'Subaru',
    'tribute': 'Mazda',
    'triple': 'Ford',
    'trooper': 'Isuzu',
    'tsx': 'Acura',
    'tucson': 'Hyundai',
    'tundra': 'Toyota',
    'ultra': 'Chevrolet',
    'ultra-low': 'Chevrolet',
    'v70': 'Volvo',
    'v90': 'Volvo',
    'vandura': 'GMC',
    'velar': 'Land Rover',
    'veloster': 'Hyundai',
    'venza': 'Toyota',
    'venture': 'Chevrolet',
    'verano': 'Buick',
    'verna': 'Hyundai',
    'versa': 'Nissan',
    'vibe': 'Pontiac',
    'viper': 'Dodge',
    'virage': 'Aston Martin',
    'vista': 'Chrysler',
    'volt': 'Chevrolet',
    'vue': 'Saturn',
    'vw': 'Volkswagen',
    'wagon': 'Ford',
    'wrangler': 'Jeep',
    'x': 'BMW',
    'x-3': 'BMW',
    'x-5': 'BMW',
    'x-7': 'BMW',
    'x-type': 'Jaguar',
    'x1': 'BMW',
    'x1/28i': 'BMW',
    'x3': 'BMW',
    'x3-28i': 'BMW',
    'x4': 'BMW',
    'x5': 'BMW',
    'x5/35i': 'BMW',
    'x6': 'BMW',
    'x6/35i': 'BMW',
    'xc40': 'Volvo',
    'xc60': 'Volvo',
    'xc70': 'Volvo',
    'xc90': 'Volvo',
    'xd': 'Scion',
    'xe': 'Jaguar',
    'xf': 'Jaguar',
    'xg': 'Hyundai',
    'xj': 'Jaguar',
    'xj-series': 'Jaguar',
    'xj6': 'Jaguar',
    'xj8': 'Jaguar',
    'xjl': 'Jaguar',
    'xjr': 'Jaguar',
    'xjs': 'Jaguar',
    'xk': 'Jaguar',
    'xk-series': 'Jaguar',
    'xk8': 'Jaguar',
    'xk8-convertible': 'Jaguar',
    'xkr': 'Jaguar',
    'xl': 'GMC',
    'xterra': 'Nissan',
    'xts': 'Cadillac',
    'xv': 'Subaru',
    'xv+cross': 'Subaru',
    'xv-crosstrek': 'Subaru',
    'yaris': 'Toyota',
    'yukon': 'GMC',
    'yukon-denali': 'GMC',
    'yukon-xl': 'GMC',
    'z3': 'BMW',
    'z4': 'BMW',
    'zephyr': 'Lincoln',
    'zoo': 'Zoo',
    'zr2': 'Chevrolet',
    'zr2.': 'Chevrolet',
    'zrx': 'Kawasaki',
    'zx2': 'Ford'
}

# Map the dictionary values to the 'manufacturer' column based on the 'model' column
null_manufacturer_rows['manufacturer'] = null_manufacturer_rows['model'].map(model_to_manufacturer)


C:\Users\Onwer\AppData\Local\Temp\ipykernel_22328\591789157.py:739: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_manufacturer_rows['manufacturer'] = null_manufacturer_rows['model'].map(model_to_manufacturer)


In [12]:
# Fill null values in 'manufacturer' column of original DataFrame
df['manufacturer'] = df['manufacturer'].fillna(null_manufacturer_rows['manufacturer'])

In [13]:
# Drop rows with null values in below columns
df = df.dropna(subset=['manufacturer', 'model','year'])


In [14]:
# Set the maximum number of null values allowed in a row
#max_null_values = 4

# Count the number of null values in each row using the isnull() method and sum() method
#null_counts_per_row = df.isnull().sum(axis=1)

# Filter out the rows with four or fewer null values using boolean indexing
#df = df[null_counts_per_row <= max_null_values]


In [15]:
# Reset the index
df.reset_index(drop=True, inplace=True)
#Reindexing DataFrame so that price feature will at last
df= df.reindex(columns=['region', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission','drive','type', 'paint_color','state', 'price'])

In [16]:
##YEAR
# Convert 'year' column from float to int
df['year'] = df['year'].astype(int)

##ODOMETER
# Calculate median
odometer_median = df['odometer'].median()
# Fill null values with the median
df['odometer'].fillna(odometer_median, inplace=True)

##Cylinders
# Split the string based on non-numeric characters and keep only the first part for 'cylinders' column
df['cylinders'] = df['cylinders'].str.split().str[0]
# Replace 'other' with nan in 'cylinders' column
df['cylinders'] = df['cylinders'].replace({'other': np.nan})

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362583 entries, 0 to 362582
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        362583 non-null  object 
 1   year          362583 non-null  int32  
 2   manufacturer  362583 non-null  object 
 3   model         362583 non-null  object 
 4   condition     223922 non-null  object 
 5   cylinders     213976 non-null  object 
 6   fuel          360377 non-null  object 
 7   odometer      362583 non-null  float64
 8   title_status  356171 non-null  object 
 9   transmission  360626 non-null  object 
 10  drive         253050 non-null  object 
 11  type          285460 non-null  object 
 12  paint_color   255894 non-null  object 
 13  state         362583 non-null  object 
 14  price         362583 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(12)
memory usage: 40.1+ MB


In [18]:
df.isnull().sum()

region               0
year                 0
manufacturer         0
model                0
condition       138661
cylinders       148607
fuel              2206
odometer             0
title_status      6412
transmission      1957
drive           109533
type             77123
paint_color     106689
state                0
price                0
dtype: int64

In [20]:
##Filling null values of condition column
# Make a copy of the DataFrame
df_copy = df.copy()

# Step 1: Split the dataset into non-null and null rows
non_null_data = df_copy.dropna(subset=['condition'])
null_data = df_copy[df_copy['condition'].isnull()]

# Step 2: Split non-null rows into features and target variable
X = non_null_data[['year', 'price', 'odometer']]
y = non_null_data['condition']

# Step 3: Train a machine learning model
model = RandomForestClassifier()
model.fit(X, y)

# Step 4: Predict the missing 'condition' values
null_X = null_data[['year', 'price', 'odometer']]
predicted_conditions = model.predict(null_X)

# Step 5: Replace missing values in the original DataFrame
df_copy.loc[df_copy['condition'].isnull(), 'condition'] = predicted_conditions

# Step 6: Evaluate the accuracy of the model
actual_conditions = non_null_data['condition']
predicted_conditions_non_null = model.predict(X)

accuracy = accuracy_score(actual_conditions, predicted_conditions_non_null)
print("Accuracy:", accuracy)



#checking for accuracy of differnt models
# List of models to try
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier()
}

# Loop over each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict the labels
    y_pred = model.predict(X_test)
    
    # Evaluate the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name}: Accuracy = {accuracy:.4f}")
#Replacing in original df
df['condition'] = df_copy['condition']


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362583 entries, 0 to 362582
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        362583 non-null  object 
 1   year          362583 non-null  int32  
 2   manufacturer  362583 non-null  object 
 3   model         362583 non-null  object 
 4   condition     362583 non-null  object 
 5   cylinders     213976 non-null  object 
 6   fuel          360377 non-null  object 
 7   odometer      362583 non-null  float64
 8   title_status  356171 non-null  object 
 9   transmission  360626 non-null  object 
 10  drive         253050 non-null  object 
 11  type          285460 non-null  object 
 12  paint_color   255894 non-null  object 
 13  state         362583 non-null  object 
 14  price         362583 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(12)
memory usage: 40.1+ MB


In [29]:
df.isnull().sum()

region               0
year                 0
manufacturer         0
model                0
condition            0
cylinders       148607
fuel              2206
odometer             0
title_status      6412
transmission      1957
drive           109533
type             77123
paint_color     106689
state                0
price                0
dtype: int64

In [48]:
# Assuming df is your DataFrame
unique_conditions = df_copy['cylinders'].unique()
print(unique_conditions)


[8 6 4 5 10 3 12]


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Separate data into two sets based on null values in the 'cylinders' column
df_non_null = df_copy[df_copy['cylinders'].notnull()]
df_null = df_copy[df_copy['cylinders'].isnull()]

# Convert non null'cylinders' to numeric type
df_non_null['cylinders'] = pd.to_numeric(df_non_null['cylinders'], errors='coerce')

# Define features and target for non-null data
X_non_null = df_non_null[['year', 'manufacturer', 'model', 'price']]
y_non_null = df_non_null['cylinders']

# Split non-null data into training and testing
X_train_non_null, X_test_non_null, y_train_non_null, y_test_non_null = train_test_split(
    X_non_null, y_non_null, test_size=0.2, random_state=42)

# Encode categorical features using target encoding
encoder = TargetEncoder(cols=['manufacturer', 'model'])
X_train_encoded = encoder.fit_transform(X_train_non_null, y_train_non_null)
X_test_encoded = encoder.transform(X_test_non_null)

# Impute missing values in encoded data
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_test_imputed = imputer.transform(X_test_encoded)

# Define classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression()
}

# Iterate through different models
for name, clf in classifiers.items():
    print(f"Evaluating Classifier: {name}")
    
    # Train the classifier
    clf.fit(X_train_imputed, y_train_non_null)

    # Predict on the test set
    y_pred = clf.predict(X_test_imputed)

    # Evaluate the accuracy
    accuracy = accuracy_score(y_test_non_null, y_pred)
    print(f"Accuracy = {accuracy:.4f}")

    print()  # Add a blank line between different classifiers


C:\Users\Onwer\AppData\Local\Temp\ipykernel_22328\403947936.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_null['cylinders'] = pd.to_numeric(df_non_null['cylinders'], errors='coerce')


Evaluating Classifier: Random Forest
Accuracy = 0.7574

Evaluating Classifier: Gradient Boosting
Accuracy = 0.6987

Evaluating Classifier: Logistic Regression
Accuracy = 0.4122



C:\Users\Onwer\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
## filling null values in cylinders
# Initialize a new target encoder
encoder_null = TargetEncoder(cols=['manufacturer', 'model'])

# Encode categorical features using target encoding for df_non_null
X_non_null_encoded = encoder_null.fit_transform(df_non_null[['year', 'manufacturer', 'model', 'price']], df_non_null['cylinders'])

# Encode categorical features using target encoding for df_null
X_null_encoded = encoder_null.transform(df_null[['year', 'manufacturer', 'model', 'price']])

# Impute missing values in encoded data
X_null_imputed = imputer.transform(X_null_encoded)

# Use the trained Random Forest model to predict missing values
predicted_cylinders = clf.predict(X_null_imputed)

# Replace null values with predicted values in df_null
df_null.loc[:, 'cylinders'] = predicted_cylinders

# Concatenate df_non_null and df_null back together
df_combined = pd.concat([df_non_null, df_null], axis=0)

# Sort the DataFrame index
df_combined.sort_index(inplace=True)

# Assign df_combined back to df_copy
df_copy = df_combined.copy()


In [44]:
df_copy.isnull().sum()

region               0
year                 0
manufacturer         0
model                0
condition            0
cylinders            0
fuel              2206
odometer             0
title_status      6412
transmission      1957
drive           109533
type             77123
paint_color     106689
state                0
price                0
dtype: int64

In [45]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362583 entries, 0 to 362582
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        362583 non-null  object 
 1   year          362583 non-null  int32  
 2   manufacturer  362583 non-null  object 
 3   model         362583 non-null  object 
 4   condition     362583 non-null  object 
 5   cylinders     362583 non-null  object 
 6   fuel          360377 non-null  object 
 7   odometer      362583 non-null  float64
 8   title_status  356171 non-null  object 
 9   transmission  360626 non-null  object 
 10  drive         253050 non-null  object 
 11  type          285460 non-null  object 
 12  paint_color   255894 non-null  object 
 13  state         362583 non-null  object 
 14  price         362583 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(12)
memory usage: 42.9+ MB


In [49]:
df['cylinders']=df_copy['cylinders']

In [50]:
df.isnull().sum()

region               0
year                 0
manufacturer         0
model                0
condition            0
cylinders            0
fuel              2206
odometer             0
title_status      6412
transmission      1957
drive           109533
type             77123
paint_color     106689
state                0
price                0
dtype: int64

In [52]:
df.to_csv('1stclean.csv',index=False)